# Prosjekt 2: Endimensjonale modeller for atomer, molekyler og krystaller
**Gruppenr:** 27 <br>
**Gruppemedlemmer:** Marte K. Høiskar, Maren Lium og Johanna U. Marstrander

## Innledning
I dette prosjektet skal vi lage en enkel modell for elektronenes oppførsel i atomer, molekyler og krystaller ved å representere atomene som potensialbrønner. 

I realiteten er atomer, molekyler og krystaller tredimensjonale systemer og elektronene vekselvirker med hverandre. Dette fører imidlertid til et svært komplekse modeller. I dette prosjektet skal vi holde oss til én dimensjon og antar at elektronene ikke vekselvirker med hverandre utover at det kun kan være to elektroner med motsatt spinn per orbital. Det vil si at maksimalt to elektroner kan ha samme romlige tilstand, her uttrykt ved bølgefunksjonen. 

Bølgefunksjonene som inneholder informasjonen om tilstanden/oppførselen til elektronene kan finnes ved å løse den tidsuavhengige Schrödingerligningen:

\begin{equation}
    \hat{\textbf{H}}\psi = E\psi
\end{equation}

Det er kun for hydrogenatomet at vi kan finne analytiske løsninger og vi må derfor løse ligningen numerisk. 

### Numerisk løsning av TUSL i en dimensjon
For en masse $m$ i en dimensjon, får den tidsuavhengige Schrödingerligningen, $\text{TUSL}$, formen:

\begin{equation}
    -\frac{\hbar^{2}}{2m}\frac{d^{2}\psi(x)}{dx^{2}}+V(x)\psi(x)=E\psi(x)
\end{equation}

Med diskretisering av operatoren for kinetisk energi får man ligningen:

\begin{equation}
    -\frac{\hbar^{2}}{2m}\frac{\psi_{n+1}-2\psi_{n}+\psi_{n-1}}{(\Delta x)^{2}}+V_{n}\psi_{n}=E\psi_{n}
\end{equation}

Vi begrenser hvor elektronet kan befinne seg ved å sette "harde vegger", det vil si $V=\infty$, på begynnelsen og enden av intervallet, ved $x=0$ og $x=L=(N+1)\Delta x$. Dermed kan vi dele opp i et endelig antall diskrete intervaller. I ligningen over blir da $\psi_{n}=\psi(n \Delta x)$, det vil si verdien av bølgefunksjonen i punktet $x_{n}$, $n$ intervaller $\Delta x$ fra startpunktet. Vi får $N$ ligninger på formen over og kan samle dem i en matriseligning:

\begin{equation}
    \bf{H}\psi = E\psi
\end{equation}

Her er $\bf{H}$ en reell og symmetrisk $N x N$ tridiagonalmatrise med elementer

\begin{equation}
    H_{nn}=\frac{\hbar^2}{m(\Delta x)^2} + V_{n},\quad H_{n,n\pm1} = -\frac{\hbar^2}{2m(\Delta x)^2}
\end{equation}

henholdsvis langs diagonalen og over og under diagonalen. 

For å finne ikke-trivielle løsninger, må

\begin{equation}
    \textrm{det}(\bf{H}-E)=0
\end{equation}

der $E$ er egenverdien ganget med identitetsmatrisen $I_{N}$. Ligningen gir $N$ løsninger for $E$: $E_{1}, E_{2}, ..., E_{N}$ med tilhørende bølgefunksjonsvektorer $\psi^{(n)}$, der $E$ og $\psi$ er henholdvis egenverdiene og egenvektorerene til matrisen $\bf{H}$. Den numeriske løsningen av Schrödingerligningen bunner i dette tilfellet dermed ut i å finne egenverdiene og egenvektorene til tridiagonalmatrisen $\bf{H}$.

For at løsningene skal danne et fullstendig sett og representere gyldige sannsynlighetsfordelinger, må de i tillegg være ortogonale og ha normert absoluttkvadrat. Det vil si at

\begin{equation}
    \sum_{n=1}^{N} {\psi_{n}^{(k)}}{\psi_{n}^{(j)}} = \delta_{kj} \quad \textrm{og} \quad
    \sum_{n=1}^{N} {|\psi_{n}^{(j)}|}^{2}  \cdot \Delta x = 1 ; \quad j,k=1,2,...,N
\end{equation}
    

### Funksjon for løsning av TUSL og noen hjelpefunksjoner
For å gjøre prosessen under selve problemløsningen enklere å gjøre og lese definerer vi først noen konstanter og lager en generell funksjon <font color='blue'> egenv </font> som finner egenverdiene og tilhørende egenvektorer/bølgefunksjoner for et gitt potensial oppdelt i gitte intervaller.

Vi innfører også tre hjelpefunksjoner: <font color='blue'> normalize </font><font color='blue'>,  get_potential_vector </font> og <font color='blue'> figur </font>. Disse sørger henholdsvis for at en gitt egenvektor/bølgefunksjon får en normert sannsynlighetsfordeling, lager en spesifisert potensialvektor og plotter hvordan en bølgefunksjon vil oppføre seg under gitte betingelser. 

In [ ]:
import numpy as np
from scipy import linalg as lg
from scipy import constants
from scipy import optimize
from matplotlib import pyplot as plt

#Definerer konstanter
m = constants.m_e        #Elektronmassen
h = constants.h          #Plancks konstant
h_b = constants.hbar     #Redusert Plancks konstant
n = 20                   #Antall intervaller per brønnområde 
pi = np.pi               #Pi
eV = constants.e         #Elektronvolt
w = 1e-9                 #Brønnbredde

<font color='blue'> egenv </font> tar inn en potensialvektor og lengden på de diskrete intervallene denne er delt opp i. Deretter lager den tridiagonalmatrisen H som beskrevet i innledningen og returnerer egenverdier og egenvektorer til denne matrisen, som utgjør den numeriske løsningen av TUSL i en dimensjon. 

In [ ]:
def egenv(V, L, n):
    delta_x = L/(n+1)
    H_d = np.array([(h_b)**2/(m*delta_x**2) + V[i] for i in range(n)])
    H_oud = np.full(n-1, -(h_b)**2/(2*m*delta_x**2))
    e_val, e_vec = lg.eigh_tridiagonal(H_d, H_oud)
    return e_val, e_vec

<font color='blue'> normalize </font> tar inn en egenvektor og lengden på intervallene og returnerer en utgave av denne som tilsvarer en sannynlighetsnormert bølgefunksjon. 

In [ ]:
def normalize(e_vec_i, delta_x ):
    return e_vec_i/np.sqrt(delta_x) 


<font color='blue'> get_potential_vector </font> tar inn antall brønner $N_{w}$, størrelsen på potensialet i brønnene $V_{0}$, bredden på brønnene $\omega$ og avstanden mellom brønnene $b$. Returverdiene er en vektor $\textbf{V}$ som representerer potensialet i hele brønnen fra 0 til L, antall målepunkter $n$ og størrelsen på hele brønnen $L$.

In [ ]:
def get_potential_vector(N_w, V_0, w, b = 0): 
    k = b/w
    
    V_s = np.zeros(10*n)                         #Område på 10w på hver side av brønnområdet
    V_w = np.ones(n) * V_0                       #Potensialet i selve brønnen/-ene
    V_b = np.zeros(int(k*n))                     #Potensialet mellom brønnene
    
    #Setter sammen V_s, V_w og V_b slik at potensialet V har riktig antall brønner,
    #avstand mellom dem og 10w på hver side av ytterste brønnområde
    V = np.append(np.append(V_s, np.tile(np.append(V_w, V_b), N_w-1)), np.append(V_w, V_s))
    
    N = 21*n + (N_w - 1) * (n + int(k*n))      #Antall punkter med verdier i V
    L = N/n * w                                #Bredden til hele "boksen"
    
    return V, N, L
   

<font color='blue'> figur </font> tar inn en posisjonsvektor **e_val**, en potensialvektor $\textbf{V}$, tittelen på plottet, størrelsen på boksen $L$ og antall målepunkter $n$. Funksjonen tar også inn en boolsk variabel som forteller om potensialet er av typen "partikkel i boks" da dette krever ekstra hensyn til grensebetingelsene. Den returnerer ingen ting, men tegner et plot som viser hvordan bølgefunksjonen vil oppføre seg under de gitte betingelsene.

In [ ]:
def figur(V, e_val, e_vec, tittel, L, partikkel_i_boks = False):
    scale = (1/np.amax(e_vec[:,0]))*1.5e-25
    
    x_val = np.linspace(0, L, len(V))
    
    delta_x = L/(len(V)+1)
    
    #Lager figur og plotter potensialet
    plt.figure(tittel, figsize=(14, 12), dpi= 80, facecolor='w', edgecolor='k')
    plt.title(tittel)
    plt.plot(x_val, V, linewidth=2, color="black")
    
    if partikkel_i_boks:
        plt.vlines(0,ymin=0, ymax=abs(e_val[maxit-1])*1.2)
        plt.vlines(L,ymin=0, ymax=abs(e_val[maxit-1])*1.2)
    
    it = 0
    
    print("Bundne tilstander: ")
    for i in range(len(e_val)):
        if e_val[i] < 0:
            vec = normalize(e_vec[:,i], delta_x)
            vec = vec * scale + e_val[i]
            print("Energinivå nr.", i+1, "=", round(e_val[i] / eV, 4), "eV")
            plt.plot(x_val, np.array([e_val[i] for x in range(len(x_val))]), "--")
            plt.plot(x_val, vec)
        else:
            if maxit == 0: break
            if it == 0 and not partikkel_i_boks: print("\nUbundne tilstander:")
            
            vec = normalize(e_vec[:,i], delta_x)
            
            if partikkel_i_boks:
                vec= np.append(np.array([0]),vec)
                vec=np.append(vec, np.array([0]))
                
            vec = vec * scale + e_val[i]
            print("Energinivå nr.", i+1, "=", round(e_val[i] / eV, 4), "eV")
            plt.plot(x_val, np.array([e_val[i] for x in range(len(x_val))]), "--")
            plt.plot(x_val, vec)
            
            it += 1
            if it == maxit:
                break
    plt.show()

## Partikkel i boks
Dersom $V(x) = 0$ for $0 < x < L$, dvs $V_{n} = 0$ for $0 < n < N + 1$, har vi en uendelig dyp potensialbrønn. Denne situasjonen kalles ofte "partikkel i boks". Dette gir enkle analysiske løsninger av TUSL slik at det er enkelt å sammenligne og verifisere at beregningene fungerer. 

### Numerisk løsning

In [ ]:
#Lager potensialvektor med n intervaller, en til å beregne egenverdier/egenvektorer og en til plotting.
V = np.zeros(n*2)
V2 = np.zeros(n*2 + 2)

#Definerer størrelsen på intervallet
L = w*2+w/n*2

#Finner numeriske egenverdier og egenvektorer
e_val_pb, e_vec_pb = egenv(V, L, 2*n)

#Bestemmer hvor mange løsninger man vil tegne
maxit = 4

#Plotter numeriske løsninger
figur(V2, e_val_pb, e_vec_pb, "Partikkel i boks", L, True)


### Analytiske løsninger
Partikkel i boks har analytiske løsninger på formen $\sqrt{2/L}\sin{k_{j}x}$. For å sjekke at funksjonene for numerisk beregning og plotting fungerer som de skal, finner og plotter vi også de analytiske løsningene i dette tilfellet. <font color='blue'> partikkel_i_boks </font> er en hjelpefunksjon for å få til dette, som tar inn hvilken løsning $(j)$ som skal returneres, lengden på området til potensialet og antall endelige punkter i vektoren som plottes til slutt. Den returnerer egenverdien og vektoren til løsning nr $j$.

In [ ]:
#funksjon som returnerer j-te egenverdi/egenvektor til analytisk løsning av TUSL for partikkel i boks
def partikkel_i_boks(j, L, N):
    delta_x = L/(N+1)
    k = j*pi/L
    an_e_val = h_b**2*k**2/(2*m)
    an_e_vec = np.array([np.sqrt(2/L) * np.sin(k*delta_x*i) for i in range(0,N+2)])
    return an_e_val, an_e_vec

#Finner og plotter analytiske løsninger
x_val = np.linspace(0, L, n*2+2)
plt.figure("Analytisk partikkel i boks", figsize=(18, 16))
plt.vlines(0,ymin=0, ymax=2.6e-19)
plt.vlines(L,ymin=0, ymax=2.6e-19)
plt.title("Analytisk partikkel i boks")
plt.plot(x_val, np.zeros(n*2+2), color="black")
for i in range(1,5):
    an_e_val_pb, an_e_vec_pb = partikkel_i_boks(i, L, n*2)
    print("Energinivå nr", i ,":", round(an_e_val_pb /eV,4), "eV" )
    plt.plot(x_val, an_e_vec_pb*6.7e-25+an_e_val_pb)
    plt.plot(x_val, np.array([an_e_val_pb for x in range(len(x_val))]), "--")
plt.show()

#### Normering og ortogonalitet
Funksjonen <font color='blue'> orthonormal_set </font> tar inn en matrise og en toleransegrense og returnerer hvorvidt kolonnene i matrisen utgjør et ortonormert sett, det vil si at vektorene er normerte og ortogonale med hverandre. <font color='blue'> orthogonal </font> er en hjelpefunksjon som tar inn to vektorer og returnerer hvorvidt de er ortogonale. 

In [ ]:
def orthogonal(v1, v2, tol):
    skalarprodukt = v1 @ v2
    return (abs(skalarprodukt)<tol)

def orthonormal_set(e_vec, tol):
    n, m = np.shape(e_vec)
    for i in range(m):
        if (not abs(e_vec[:,i]@e_vec[:,i] - 1)<tol):                       #Sjekker at vektorene er normerte
            return False
        for j in range(m):
            if (j != i and (not orthogonal(e_vec[:,i], e_vec[:,j], tol))): #Sjekker at ulike egenvektorer er ortogonale
                return False
    return True

#Sjekker hvorvidt egenvektorene til partikkel-i-boks-tilfellet er ortogonale
print(orthonormal_set(e_vec_pb, 0.001))

### Svar på oppgave 1: 
Utregningene viser at de numeriske beregnede egenverdiene stemmer relativt godt overens med den analytiske løsningen. Et mer nøyaktig resultat kan oppnås ned å dele opp i flere mindre intervaller. De numeriske løsningene stemmer også godt med teorien, der grunntilstanden ikke har noen nullpunkter, 1. eksiterte tilstand har ett nullpunkt og så videre. Løsningene er annenhver gang symmetrisk og antisymmetriske om midtpunktet. Ved hjelp av funksjonen <font color='blue'> orthonormal_set </font> ser vi også at de numerisk beregnede egenfunksjonene er ortonormale og de danner dermed et fullstendig sett. 

## Atomer
En endimensjonal modell med en enkelt potensialbrønn kan brukes til å modellere et atom. Bølgefunksjonene og energinivåene denne modellen gir vil kvalitativt beskrive tilstanden til elektronene rundt atomet, både diskrete bundne tilstander og et kontinuerlig spektrum av ubundne tilstander som representerer frie elektroner. 

### Oppgave 2a
Vi bruker en endelig potensialbrønn med dybde $V_{0}$ og bredde $\omega$ som modell for et atom, der vi inkluderer et område med potensial $V=0$ og bredde $10\omega$ på hver side. Med de harde veggene ($V = \infty$) i god avstand fra brønnen forventer vi at de numerisk begregnede bundne tilstandene stemmer bra overens med den analytiske løsningen, men uten harde vegger i $x = 0$ og $x = L$. Med $\omega = 1 \cdot 10^{-9}$ m må vi finne hvilke verdier av $V_{0}$ som gir tre bundne tilstander. I cellen under er det definert en funksjon <font color='blue'> N </font> som tar inn et potensial og returnerer antall bundne tilstander dette vil gi i en potensialbrønn med det gitte potensialet. Deretter brukes <font color='blue'> scipy.optimize.fsolve </font> til å regne ut det aktuelle intervallet av $V_{0}$ for 3 bundne tilstander. Til slutt definerer vi en konstant $V_{0}$ fra dette intervallet som skal representere brønndybden videre. 

In [ ]:
def N(V_0):
    return 1 - n_bundne + ((w*np.sqrt(2 * m * V_0))) / (np.pi * h_b)

n_bundne = 3
root_3 = optimize.fsolve(N, np.array(1E-19))     #Nedre potensialnivå, lavere potensialer gir kun 2 bundne tilstander eller færre

n_bundne = 4
root_4 = optimize.fsolve(N, np.array(1E-19))     #Øvre potensialnivå, høyere potensialer gir 4 bundne tilstander eller flere

print(f"Med en omega = {w}m får man {n_bundne - 1} bundne tilstander med et potensial i intervallet [ {float(root_3)/eV:.6} eV , {float(root_4)/eV:.6} eV ].")


Vi definerer en konstant $V_{0}$ fra dette intervallet som skal representere brønndybden videre. Deretter oppretter vi en potensialvektor for tilfellet i oppgaven og plotter resultatet. 

In [ ]:
#Definerer brønndybde
V_0 = -4e-19

#Lager potensialvektor og henter antall målepunkter og størrelse på boksen
V_a, N_a, L_a = get_potential_vector(1, V_0, w)

#Finner egenverdier og egenvektorer for atom
e_val_a, e_vec_a = egenv(V_a, L_a, N_a)
#Bestemmer hvor mange ubundne løsninger man vil tegne
maxit = 10

#Plotter figuren for atom
figur(V_a, e_val_a, e_vec_a, "Atom", L_a)

#### Er løsningene fornuftige?
Fra plottet ser vi at løsningene er annenhver gang symmetriske og antisymmetriske (grunntilstanden er symmetrisk) omkring brønnens midtpunkt og at de ulike energinivåene medfører økende antall nullpunkter (grunntilstanden har ingen nullpunkter). Dersom vi plotter mange ubundne tilstander, ser vi at disse danner et tilnærmet kontinuerlig spektrum for $E > 0$. Dette stemmer overens med teorien. For å undersøke hvorvidt løsningene har fornuftige egenskaper utover dette, sammenligner vi våre numerisk beregnede egenverdier med de teoretiske løsningene og undersøker tilstandenes bølgelengde. 

##### Teoretiske energier
Dersom man har en endelig potensialbrønn med bredde $L = 2l$ og dybde $V_{0}$ gir den analytiske løsningen egenverdier på formen

\begin{equation}
   tan( z ) = \left\{ 
   \begin{array}{ll}
       \left[ \left ( \frac{z_{0}}{z} \right )^{2} - 1 \right] ^{\frac{1}{2}} & \text{(symmetriske løsninger)} \\
       - \left[ \left ( \frac{z_{0}}{z} \right )^{2} - 1 \right] ^{-\frac{1}{2}}& \text{(antisymmetriske løsninger)} \\
   \end{array}
   \right.
\end{equation}

med $z = \sqrt{2m(V_0-E)l^{2}}/ \hbar$ og $z_{0} = \sqrt{2mV_{0}l^2} / \hbar$.


Nedenfor har vi løst dette likningssettet numerisk for å finne løsningene tilsvarende vårt problem ved bruk av hjelpefunksjonene i cellen under. Vi har satt $L$ og $V_{0}$ som ovenfor.

In [ ]:
z0 = np.sqrt(2*m*abs(V_0)*(w/2)**2) / h_b       #z0 som gitt i teksten ovenfor
 
#Regner ut energien ved en gitt z-verdi (der z er gitt som i teksten ovenfor)
def energi(z):  
    return -abs(V_0) + z**2 * h_b**2 / (2 * m * (w/2)**2)
   
#Den symmetriske delen til høyresiden av funksjonen ovenfor
def symmetrisk(z):                      
    return np.sqrt((z0/z)**2 - 1)

#Den antisymmetriske delen til høyresiden av funksjonen ovenfor
def anti_symmetrisk(z):                     
    return -1 / np.sqrt((z0/z)**2 - 1)

#Funksjon som gir output 0 når tan(z) og den symmetriske løsningen skjærer hverandre. Brukes til å finne skjæringspunktet
def skjaer_symmetrisk(z):
    return np.tan(z) - symmetrisk(z)

#Funksjon som gir output 0 når tan(z) og den antisymmetriske løsningen skjærer hverandre. Brukes til å finne skjæringspunktet
def skjaer_antisymmetrisk(z):
    return np.tan(z) - anti_symmetrisk(z)


z = np.arange(1, 4000) * 1E-3                 #Lager et array med verdier for z mellom 0 og 1

tan = np.tan(z)                               #Array med løsninger tan(z) 
s = symmetrisk(z)                             #Array med symmetriske løsninger for z
a_s = anti_symmetrisk(z)                      #Array med antisymmetriske løsninger for z

threshold = 100                               #Fjerner asymptoten til tan fra plottet
tan[tan < - threshold] = np.inf

plt.plot(z, tan, 'b-', label='tan(z)')                                 #Plotter tan(z)
plt.plot(z, s, 'r-', label='symmetriske løsninger')                    #Plotter de symmetriske verdiene
plt.plot(z, a_s, 'g-', label='antisymmetriske løsninger')              #Plotter de antisymmetriske verdiene

idx_s = np.argwhere(np.diff(np.sign(tan - s))).flatten()               #Finner indeksene til de symmetriske skjæringspuntene
idx_as = np.argwhere(np.diff(np.sign(tan - a_s))).flatten()            #Finner indeksene til de antisymmetriske skjæringspunktene

plt.plot(z[idx_s], tan[idx_s], 'ro')                                   #Markerer skjæringspunktene på plottet
plt.plot(z[idx_as], tan[idx_as], 'ro')

#Markerer resten av elementene i plottet
plt.ylim(-10, 10)                                                   
plt.title("Bundne løsninger med endelig brønndybde $V_{0}$")           
plt.xlabel(r"$z$")
plt.legend()


#Regner ut skjæringspunktene og legger dem i et array
z_total = np.array([])

z_1 = optimize.fsolve(skjaer_symmetrisk, 1)
z_total = np.append(z_total, np.array([z_1]))
z_2 = optimize.fsolve(skjaer_antisymmetrisk, 2.5)
z_total = np.append(z_total, np.array([z_2]))
z_3 = optimize.fsolve(skjaer_symmetrisk, 3.5)
z_total = np.append(z_total, np.array([z_3]))

#Bruker energi(z)-funksjonen til å regne ut energiegenverdiene fra z-verdiene
E = energi(z_total)

print("En potensialbrønn med endelig dybde V0 =", V_0, "J og bredde L =", w, "m gir energiene:\n")

for i in range(len(E)):
    print("Energinivå nr.", i+1, "=", f"{E[i]/eV : .4} eV | Numerisk: {e_val_a[i] /eV:.4} eV | Forskjell:", 
          f"{(e_val_a[i] - E[i])/eV:.4}eV    {(1 - e_val_a[i] / E[i]) * 100:.5}%" )

plt.show()

Her ser man at de numerisk beregnede egenverdiene stemmer godt overens med de teoretiske løsningene. Med tre bundne tilstander der $\omega = 1 \cdot 10^{-9}$ m og $V_{0} = -4 \cdot 10^{-19}$ J ser man at den største feilen er på 2.7% for energinivå 3 og under 0.4% for energinivå 1 og 2. 

##### Bølgelengde
<font color = 'blue'> wavelength </font> beregner bølgelengden til en gitt tilstand med total energi $E$ og potensiell energi $V_0$. Bølgelengden $\lambda$ er gitt ved $\lambda = \frac{2\pi}{k}$ der bølgetallet er $k = \sqrt{2m(E-V_0)}$. 


In [ ]:
delta_x_a = L_a/(N_a+1)

def wavelength(E, V_0):
    wave_num = np.sqrt(2*m*abs(E-V_0))/h_b             #Bølgetall
    wave_length = 2*pi/wave_num                        #Bølgelengde beregnet med bølgetallet
    print("Wave length =", wave_length, "m")


#Bundne tilstander
print("Bundne tilstander")
for i in range(3):
    print("\nTilstander nr.", i, "i brønnen:")
    wavelength(e_val_a[i], V_0)
    print("Bølgelengde til partikkel i boks:", w*(2/(1+i)))
        
#Ubundne tilstander
#Utenfor brønnen
print("\nUbunden tilstand")
print("Utenfor brønnen:")
wavelength(e_val_a[4], 0)

#I brønnen
print("\nI brønnen:")
wavelength(e_val_a[4], V_0)

I følge teorien skal bølgefunksjon nr. $j$ til en partikkel i boks ha en bølgelengde $\lambda = 2\cdot w/(1+j)$. Ettersom vi modellerer et atom ved å lage en endelig dyp potensialbrønn vil bølgefunksjonen gå inn i det klassiske forbudte området. Dermed blir bølgelengden litt lenger enn bølgelengden for en partikkel i boks. Dette ser man stemmer for de beregnede verdiene av bølgelengden til de tre bundne tilstandene.

Ut fra teorien er den kinetiske energien høyere i brønnen enn utenfor. Dette stemmer overens med plottet av bølgefunksjonene til et atom. Bølgelengden kan uttrykkes ved den kinetiske energien $K$ som 

\begin{equation}
  \lambda = \frac{2\pi}{\sqrt{2mK}.}
\end{equation}

Da må bølgelengden være kortere i brønnen enn utenfor. Den ubundne tilstanden med lavest energi har $\lambda = 0.78$ nm i brønnen, mens utenfor brønnen er $\lambda = 19.8$ nm. Dermed stemmer de beregnede verdiene overens med teorien.

### Oppgave 2b
I følge Paulis prinsipp kan ikke to identiske fermioner okkupere samme kvantetilstand. Derfor kan det bare eksistere to elektroner med ulikt spinn i hver bundne tilstand. Med tre bundne tilstander kan man maksimalt ha seks elektroner. De aktuelle atomnumrene er da $1, 2, 3, 4, 5$ og $6$. Like og odde atomnumrene vil henholdsvis ha totalt elektronspinn lik  $0$ og $\pm 0.5$  i grunntilstanden.

## Molekyler
Dersom vi setter sammen flere potensialbrønner ved siden av hverandre, får vi en modell for molekyler der flere atomer har slått seg sammen. Hver "brønn" vil representere et atom, mens avstanden mellom dem tilsvarer bindingslengden til molekylet. Vi ser kun på homonukleære atomer; da er alle potensialbrønnene like dype. Modellen kan for eksempel beskrive hydrogengass der to hydrogenatomer har bundet seg sammen. I grunntilstanden kan man se for seg at to elektroner, et fra hvert hydrogenatom, vil befinne seg i orbitalen som svarer til den laveste energiegenverdien, jamfør Paulis prinsipp. Dersom modellen i stedet skal beskrive Heliumgass, $\text{He}_{2}$, vil to elektroner fylle orbitalen tilsvarende den laveste energiegenverdien, mens to elektroner vil fylle neste orbital. 

### Oppgave 3
I cellen under ser vi på et homonukleært toatomig molekyl med bindinglengde $b$. 

In [ ]:
#Avstand mellom brønnene/atomene
b = w/2

#Lager potensialvektor og henter antall målepunkter og størrelse på boksen
V_m, N_m, L_m = get_potential_vector(2, V_0, w, b)

#Finner egenverdier og egenvektorer
e_val_m, e_vec_m = egenv(V_m, L_m, N_m)

#Bestemmer for mange ubundne løsninger man vil tegne
maxit = 0

#Plotter figuren for toatomig molekyl
figur(V_m, e_val_m, e_vec_m, "Molekyl", L_m)

#### Bindingsenergi
Bindingsenergi er energimengden som trengs eller frigjøres når to atomer går sammen og danner et molekyl. Det vil si at det er energiforskjellen mellom de atomære og molekylære energinivåene. Funksjonen <font color='blue'> bindingsenergi </font> tar inn atomnummeret til atomtypen som går sammen og danner et molekyl og to vektorer med egenverdier til henholdsvis atomet og molekylet. Den returnerer bindingenergien, der negativ energi svarer til frigjort energi ved å binde seg og positiv energi svarer til energi som eventuelt må tilføres for at atomene skal binde seg. 

Til slutt beregnes bindingsenergien til hydrogen- og heliumgass. 

In [ ]:
def bindingsenergi(atom_num, eval_at, eval_mol):
    b_e = 0
    orbital_type = 0
    for i in range(atom_num):
        term = eval_mol[i]-eval_at[orbital_type] #Finner energiforskjellen mellom atomære og molekylære energinivåer
        b_e += term
        if (i+1)%2 == 0:
            orbital_type += 1
    return b_e/eV

#Bindingsenergi til Hydrogengass
b_e_h2 = bindingsenergi(1, e_val_a, e_val_m)
print("Bindingsenergi til H2:", b_e_h2, "eV")

#Bindingsenergi til Heliumgass
b_e_h2 = bindingsenergi(2, e_val_a, e_val_m)
print("Bindingsenergi til He2:", b_e_h2, "eV")
    

Med $V_{0} = -2.5\textrm{ eV}$, $w = 1\textrm{ nm}$ og $b = 0.5\textrm{ nm}$ er bindingsenergien til $\textrm H_{2}$ og $\textrm {He}_{2}$ henholdsvis -0.0021 eV og -0.00016 eV. Her er negativ energi definert som frigjort energi og positiv energi definert som tilført energi. Det vil si at når hydrogengass dannes vil det frigjøres energi, mens det må tilføres energi for å danne heliumgass. Når to like atomer reagerer og danner et molekyl vil molekylet ha to energinivåer der de to energinivåene ligger henholdsvis over og under atomets energinivå (se figur). Det laveste energinivået representerer en bindende orbital, mens det høyeste energinivået tilhører en antibindende orbital. I hydrogengass fylles kun det laveste energinivået siden det er plass til opptil 2 elektroner i følge Paulis prinsipp og hydrogenatomene har ett elektron hver. Dermed skal det i teorien frigjøres energi når $\text{H}_{2}$ dannes, noe som stemmer med vår modell. For $\text{He}_{2}$ vil derimot både den bindende og antibindende orbitalen fylles ettersom heliumatomene har to elektroner hver. Et prinsipp i molekylorbitalteori er at antibindende orbitaler er mer antibindende enn bindende orbitaler er bindende. Det vil si at energiforskjellen mellom molekylets høyeste energinivå og atomets energinivå er større enn energiforskjellen mellom molekylets laveste energinivå og atomets energinivå. Dermed vil det måtte tilføres energi for å danne $\text{He}_{2}$, og vi finner heller ikke $\text{He}_{2}$ naturlig. Dette stemmer ikke overens med vår modell fordi den beregnede bindingsenergien til $\text{He}_{2}$ er negativ, som vil si at det frigjøres energi ved dannelse av heliumgass. Likevel kan man observere at bindingsenergien til $\text{He}_{2}$ er mindre enn bindingsenergien til $\text{H}_{2}$ noe som tyder på at bindingen mellom heliumatomene er svakere enn mellom hydrogenatomene. Dette stemmer overens med teorien.

<img src="bilde.png">

## Krystaller
Ved å utvide modellen for et molekyl med et stort antall potensialbrønner kan vi få en endimensjonal modell for en krystall. 

In [ ]:
#Setter antall brønner og avstand mellom brønnene 
N_w = 12
b = w/4

#Lager potensialvektor og henter antall målepunkter og størrelse på boksen
V_cr, N_cr, L_cr = get_potential_vector(N_w, V_0, w, b)

#Finner energiegenverdier og egenvektorer
e_val_cr, e_vec_cr = egenv(V_cr, L_cr, N_cr)

#Plotter figuren for krystall
figur(V_cr, e_val_cr, e_vec_cr, "Krystall", L_cr)

### Oppgave 4a
Ved å plotte vises det tydelig at dersom $N_{w} = 5$ eller $N_{w} = 10$ blir det henholdsvis $5$ og $10$ energinivåer omkring hvert atomære energinivå. Mer generelt kan man si at med $N_{w}$ atomer dannes det $N_{w}$ energinivåer omkring hvert atomære energinivå. 

### Oppgave 4b
Disse $N_{w}$ energinivåene rundt hvert energinivå strekker seg over et energiområde som kalles *energibånd*. Med et stort antall brønner (og dermed energinivåer) får man en tilnærmet kontinuerlig fordeling av energinivåer innenfor hvert energibånd. Bredden på et slikt bånd finner man ved å se på differansen mellom det høyeste og laveste energinivået blant de $N_{w}$ energinivåene rundt hvert atomære energinivå. Her vil vi se på båndbredden til energinivåene omkring de energinivåene til de tre bundne tilstandene for utvalgte $N_{w}$ mellom 2 og 100. 

<font color = 'blue'>band_w_plot</font> plotter båndbredden til disse tre energibåndene som funksjon av antall atomer $N_{w}$ i krystallen, og gir båndbredden ved det tilfellet med flest brønner på plottet.

In [ ]:
def band_w_plot():
    #Lager figur 
    tittel = "Båndbredde som funksjon av antall atomer"
    plt.figure(tittel)

    x_values = np.array([i for i in range(2, 101, 3)])     #Verdier for N_w
    
    band_bredde_gr = np.array([])                          #Båndbredde rundt grunntilstanden
    band_bredde_1 = np.array([])                           #Båndbredde rundt 1. eksiterte tilstand 
    band_bredde_2 = np.array([])                           #Båndbredde rundt 2. eksiterte tilstand
    
    for n_w in x_values:
        v_temp, N_temp, L_temp = get_potential_vector(n_w, V_0, w, b)     #Finner potensial                               
        e_val_temp, e_vec_temp = egenv(v_temp, L_temp, N_temp)            #Henter ut energiegenverdiene (og vektorene, men de trenger vi ikke)
        
        band_bredde_gr = np.append(band_bredde_gr, np.array([e_val_temp[n_w - 1] - e_val_temp[0*n_w]]))   #Finner båndbredden til grunntilstanden men n_w brønner
        band_bredde_1 = np.append(band_bredde_1, np.array([e_val_temp[2*n_w - 1] - e_val_temp[1*n_w]]))   #Finner båndbredden til 1. eksiterte tilstand men n_w brønner
        band_bredde_2 = np.append(band_bredde_2, np.array([e_val_temp[3*n_w - 1] - e_val_temp[2*n_w]]))   #Finner båndbredden til 2. eksiterte tilstand men n_w brønner

    plt.plot(x_values, band_bredde_gr)      #Plotter båndbreddene
    plt.plot(x_values, band_bredde_1)
    plt.plot(x_values, band_bredde_2)
    
    plt.title(tittel)
    plt.xlabel("N_w")
    plt.ylabel("E / ( J )")
    
    plt.text(75, band_bredde_gr[-1] * (1 + 0.14), r"$E_{1} =$" + str(round(band_bredde_gr[-1]*1e20,2)) + "e-20 J")
    plt.text(75, band_bredde_1[-1] * (1 + 0.05), r"$E_{2} =$" + str(round(band_bredde_1[-1]*1e20,2)) + "e-20 J")
    plt.text(75, band_bredde_2[-1] * (1 - 0.05), r"$E_{3} =$" + str(round(band_bredde_2[-1]*1e20,2)) + "e-20 J")
    
    plt.show()
    
band_w_plot()

Fra plottet over kan man se at båndbreddene raskt nærmer seg bestemte verdier når $N_{w}$ øker. Disse verdiene er oppført i plottet. 


### Oppgave 4c

Med $N_w$ brønner ($N_w$ atomer) og 4 elektroner per atom, vil elektronene fylle opp de 2$N_w$ orbitalene med lavest energi når de er i grunntilstanden. Avstanden mellom energien til de to elektronene med høyest energi i denne krystallen og opp til laveste energinivå i neste ledige energibånd (som ved svært lave temperaturer er tomt) kalles *båndgap*. Størrelsen på dette båndgapet bestemmer om krystallen er en leder, halvleder eller isolator. Er båndgapet lik null har vi en leder, dersom det er større enn null men mindre enn et par tre eV har vi en halvleder, og dersom det er større enn dette er krystallen en isolator.

I vårt tilfelle får vi følgenden verdier for energinivået til de to elektronene med høyest energi og båndgap: 

In [ ]:
print("Høyeste energi til elektronene er: ", round(e_val_cr[2 * N_w - 1]/eV , 5), " eV")
print("Båndgapet er på: ", round((e_val_cr[2*N_w] - e_val_cr[2 * N_w - 1])/eV, 4), " eV.")

Dette båndgapet tyder på at vår krystall er en halvleder.

### Oppgave 4d
Løsningen av TUSL i et periodisk potensial skal ha formen

\begin{equation}
    \psi(x) = e^{ikx}u_{k}(x) \ \ \ \text{med} \ \ \ \space u_{k}(x+a) = u_{k}(x).
\end{equation}

I følge Blochs teorem skal $a$ være lik periodisiteten til gitteret. I cellen under har vi definert funksjonen <font color='blue'> Bloch_plott </font> som plotter og regner ut periodisiteten til Bloch-funksjonene til egenfunksjonene. 

In [ ]:
def Bloch_plott():
    
    #Lager en figur
    tittel = "Bloch-plott"
    plt.figure(tittel, figsize=(18, 5), dpi= 80, facecolor='w', edgecolor='k')

    it = 3    #Antall energinivåer som skal vurderes
    
    x_val_cr = np.linspace(0, L_cr, N_cr)  
    dx_cr = L_cr / (N_cr + 1)
    
    for i in range(it):
        e_vec_cr_norm = normalize(e_vec_cr[:,i], dx_cr)                             #Normaliserer egenfunksjonen
        
        print("Energinivå nr.", i+1, "=", round(e_val_cr[i] / eV, 4), "eV")
        
        plt.plot(x_val_cr, e_vec_cr_norm)                                           #Plotter egenfunksjonen
        
        grad = np.gradient(e_vec_cr_norm)                                           #Deriverer egenfunksjonen
        idx = np.argwhere(np.diff(np.sign(grad - np.zeros(len(grad))))).flatten()   #og finner indeksene til nullpunktene

        #plt.plot(x_val_cr[idx], e_vec_cr_norm[idx], 'o')
        #plt.plot(x_val_cr, grad)
         
        a = x_val_cr[idx[int(len(idx)/(i+2)) + 1]] - x_val_cr[idx[int(len(idx)/(i+2)) - 1]]   #beregner perioden til u(x)
        
        print("Periodisiteten til u%d(x) = u%d(x+a): a ="% (i+1, i+1), f"{a:.6}", "m | Forskjell:", f"{w+b-a:.6}", 
              "m    ", f"{(1 - (w+b)/a) * 100:.4}" , "%")
        
        print()
    print("(Forskjellen er regnet ift. w+b = 1.25e-9 m)")
    
    plt.show()
    
Bloch_plott()

Potensialet vårt er periodisk med gitterkonstanten $a = \omega +b = 1.25 \text{nm}$. De beregnede verdiene for periodisiteten til våre bølgefunksjoner omtrent lik den teoretiske verdien, med en feil på 0.1439% for $\omega = 1$ nm, $b = \omega / 4$ og $V_0 = -4 \cdot 10^{-19} J$. 
